##### Author = Trung Huynh, huynh.trung@mayo.edu

# Step A. Aggregates all csv files. Ensure script is in same file as csv's.

In [ ]:
import pandas as pd
import glob
import os
from datetime import datetime # 1. Import datetime

# --- Configuration ---
metric_map = {
    "meanFullTransientAmplitude": "Amp",
    "meanBPMorZero": "BPM",
    "meanDecayTime": "Decay",
    "meanRiseTime": "Rise",
    "meanUpstrokeVelocity": "Up_Velocity",
    "td50": "TD50",
    "td90": "TD90",
    "cvFullTransientPeriod": "CV_Per",
    "td50Normalized": "TD50_Norm", # Added normalized metric
    "td90Normalized": "TD90_Norm",  # Added normalized metric
    "meanMinValue": "MinValue"
}

# Define which metrics require Rate Correction (Linear Regression vs Beat Period).
# Only these metrics will include the X-axis (Period) in the output.
rate_dependent_metrics = [
    "meanDecayTime", 
    "meanRiseTime",
    "meanUpstrokeVelocity", 
    "td50", 
    "td90"
]

prefixes = ["Calcium API F(t)", "Calcium Stretch F(t)"]
period_col_suffix = "(meanFullTransientPeriod)"

# --- Processing ---
files = glob.glob("*.csv")

# Store data as a dictionary of dictionaries (Sheet -> {Column: Series})
data_collectors = {} 

print(f"Found {len(files)} CSV files. Processing...")

for file in files:
    # 1. Identify Group Name
    try:
        # Splits "Table_R541C_..." to get "R541C"
        group_name = os.path.basename(file).split('_')[1]
    except IndexError:
        group_name = os.path.splitext(os.path.basename(file))[0]
    
    # 2. Find Header
    header_idx = 0
    with open(file, 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("id,Well"):
                header_idx = i
                break
    
    # 3. Load Data
    try:
        df = pd.read_csv(file, skiprows=header_idx)
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue
    
    # 4. Extract Metrics
    for prefix in prefixes:
        x_col_name = f"{prefix} {period_col_suffix}"
        
        # Check if the prefix data exists in this file (by checking for the period column)
        if x_col_name not in df.columns:
            continue 
            
        x_data = df[x_col_name]
        
        for metric, abbr in metric_map.items():
            col_name = f"{prefix} ({metric})"
            
            if col_name in df.columns:
                func_type = prefix.split()[1] # "API" or "Stretch"
                sheet_name = f"{func_type}_{abbr}"
                
                # Initialize sheet dictionary if new
                if sheet_name not in data_collectors:
                    data_collectors[sheet_name] = {}
                
                # Logic: Only include X (Period) if the metric is rate-dependent
                if metric in rate_dependent_metrics:
                    data_collectors[sheet_name][f"{group_name}_X"] = x_data.reset_index(drop=True)
                
                # Always store the Y (Metric) value
                data_collectors[sheet_name][f"{group_name}_Y"] = df[col_name].reset_index(drop=True)

# --- Save to Excel ---
# Get today's date formatted as MMDDYEAR
date_suffix = datetime.now().strftime("%m%d%Y")

output_file = f"Prism_Ready_Data_{date_suffix}.xlsx"
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, columns_dict in data_collectors.items():
        # Create DataFrame from the dictionary of Series
        df_sheet = pd.DataFrame(columns_dict)
        
        # Sort columns to keep X and Y pairs together (e.g. Control_X, Control_Y)
        # For metrics without X, this will just sort the Y columns alphabetically.
        df_sheet = df_sheet.reindex(sorted(df_sheet.columns), axis=1)
        
        safe_name = sheet_name[:31]
        df_sheet.to_excel(writer, sheet_name=safe_name, index=False)

print(f"Done! Data saved to '{output_file}' with full row counts.")

Found 4 CSV files. Processing...
Done! Data saved to 'Prism_Ready_Data_Fixed.xlsx' with full row counts.


# Step B: Comparison of Means (One-way ANOVA)
For the metrics where rate correction is not required (e.g., Amplitude, BPM, CV) - only need _Y column from output aggregate excel:

1. New Data Table: Select Column graph type.

2. Data Entry: Paste your data into columns: Control, R541C, H222P, R225X.

3. Analyze: Click Analyze > One-way ANOVA (or Kruskal-Wallis if data is non-normal).

- Multiple Comparisons: Choose "Compare the mean of each column with the mean of a control column" (Dunnett's test). Select your Isogenic Control as the reference.

- Output: This will give you significance flags (astrisks) for every variant against the control.

# Step C: Rate Correction (Regression Analysis)
For the 4 rate-dependent metrics (meanDecayTime, meanRiseTime, td50, td90):

1. New Data Table: Select XY graph type.

2. Data Entry:

- X Values: Paste meanFullTransientPeriod (Beat Period) here.

- Y Values: Create Group columns for Control, R541C, H222P, R225X.

- Importantly: You must match the X (Period) and Y (Metric) for each individual cell. In Prism, you will list all X-values in the X column, and the corresponding Y-values in the specific Group column.

3. Analyze: Click Analyze > Simple Linear Regression (or Nonlinear if you have a specific correction formula like Fridericia).

- Compare: Check the box "Test whether the slopes and intercepts are significantly different".

4. Interpretation:

- Different Slopes: The variant reacts differently to changes in beat rate than the control.

- Different Intercepts (same slope): The variant has a fundamentally higher/lower value (e.g., prolonged decay) regardless of beat rate.

5. Visualization: This creates the plot you requested: Metric (Y) vs Period (X), with regression lines color-coded by group.

Each tab (e.g., API_meanDecayTime) corresponds to one analysis.

For XY plots: Copy the Control_X, Control_Y columns into Prism's XY table (Prism allows importing X/Y pairs or shared X; for unpaired cells with different periods, use "X" and "Y" columns for each group, or stack them: One X column, Multiple Y columns, but you must align rows correctly. Actually, Prism XY tables usually expect one X column. Since every cell has a different beat period, you should format the Excel such that you paste [Period, Value] for Control, then [Period, Value] for R541C below it, utilizing a "Group" column if importing, OR use Prism's "Multiple Variables" format.)

Easier Prism XY Method: Paste X (Period) and Y (Metric) for Control into the first 2 columns. Then X and Y for R541C into the next 2 columns. Prism treats these as separate datasets on the same graph.